***
# **Exercise 5: Stack Application - Conversion from Infix to Postfix**

***
### [**John Mike Asuncion**](https://github.com/johnmikx)
![BSCPE 2–2](https://img.shields.io/badge/BSCPE_2–2-18BCF2?style=for-the-badge&logo=Home%20Assistant&logoColor=white)
![CMPE 201 DSA](https://img.shields.io/badge/[CMPE_201]_Data_Structures_and_Algorithms-FF3621?style=for-the-badge&logo=Databricks&logoColor=white)

#### **Professor: Engr. Godofredo T. Avena**
##### *October 22, 2025*
***

## **Table of Contents**

* [**1. Objective & Notes**](#1)
* [**2. Key Algorithm Idea**](#2)
* [**3. Stack Template (Node + Stack)**](#3)
* [**4. Infix to Postfix Conversion**](#3)
* [**5. Examples & Expected Outputs**](#3)
***

## **1. Objective & Notes**  <a class="anchor" id="1"></a>

**Objective**: convert infix expressions to postfix expressions observing operator precedence and parentheses. Use a stack to temporarily hold operators.

**Notes**:

- Operator precedence: `^` (right assoc) > `*` `/` (left) > `+` `-` (left).
- The `Stack` class from the provided `stack_template.py` is used as-is (pushed/pop/peek).

## **2. Key Algorithm Idea**  <a class="anchor" id="2"></a>

Walk the expression token by token:

* If token is operand → append to output.
* If token is `'('` → push to stack.
* If token is `')'` → pop operators to output until `'('`.
* If token is operator → pop from stack to output while top of stack has operator with **higher precedence** (or equal precedence and operator is **left-associative**). Then push current operator.
* After reading input, pop remaining operators to output.

Refer to `lab_5.md` for examples and precedence rules.

## **3. Stack Template (Node + Stack)**  <a class="anchor" id="3"></a>

In [36]:
##########################
# This `stack_template.py`
# provided by our prof.
##########################

class Node:
    def __init__(self, data):
        self.data = data
        self.next = None


class Stack:
    def __init__(self):
        self.top = None

    def push(self, data):
        new_node = Node(data)
        if self.top:
            new_node.next = self.top
        self.top = new_node

    def pop(self):
        if self.top is None:
            return None
        else:
            popped_node = self.top
            self.top = self.top.next
            popped_node.next = None
            return popped_node.data

    def peek(self):
        if self.top:
            return self.top.data
        else:
            return None

    def is_empty(self):
        return self.top is None

    def print_stack(self):
        if self.top is None:
            print("Stack is empty")
        else:
            current = self.top
            print("Stack elements (top → bottom):")
            while current:
                print(current.data)
                current = current.next

## **4. Infix to Postfix Conversion**  <a class="anchor" id="4"></a>

In [43]:
#####################################
# This is infix to postfix conversion
# using the Stack above.
#####################################

def is_operand(ch):
    """Return True if ch is an operand (letter or digit)."""
    return ch.isalnum()  # letters and digits

def precedence(op):
    """Return precedence number: higher means higher precedence."""
    if op == '^':
        return 3
    if op in ('*', '/'):
        return 2
    if op in ('+', '-'):
        return 1
    return 0

def is_right_associative(op):
    """'^' is right associative; others considered left by default."""
    return op == '^'

def infix_to_postfix(expr):
    """
    Convert an infix expression (string) into a postfix expression (string).
    Assumptions:
    - Tokens can be single-letter operands or digits (e.g., A, B, 1, 2).
    - Operators: +, -, *, /, ^
    - Parentheses: (, )
    - Spaces are allowed and ignored (we'll split by characters but skip spaces).
    """
    ops = Stack()
    output_tokens = []

    # Iterate through characters (skip spaces)
    for ch in expr:
        if ch == ' ':
            continue
        if is_operand(ch):
            # operand -> output
            output_tokens.append(ch)
        elif ch == '(':
            ops.push(ch)
        elif ch == ')':
            # pop until '(' or stack empty
            while not ops.is_empty() and ops.peek() != '(':
                output_tokens.append(ops.pop())
            # pop the '(' itself
            if not ops.is_empty() and ops.peek() == '(':
                ops.pop()
            else:
                # mismatched parenthesis, ignore for now
                pass
        else:
            # operator
            # pop while top of stack has operator with higher prec
            # or equal prec and current op is left-associative
            while (not ops.is_empty() and ops.peek() != '(' and
                   ((not is_right_associative(ch) and precedence(ops.peek()) >= precedence(ch))
                    or (is_right_associative(ch) and precedence(ops.peek()) > precedence(ch)))):
                output_tokens.append(ops.pop())
            ops.push(ch)

    # pop remaining operators
    while not ops.is_empty():
        top = ops.pop()
        if top in ('(', ')'):
            # skip any stray parentheses
            continue
        output_tokens.append(top)

    # Return space-separated tokens as conventional postfix
    return ' '.join(output_tokens)

## **5. Examples & Expected Outputs**  <a class="anchor" id="5"></a>

In [51]:
examples = [
    ("(A+B+C)*D", "expected: A B + C + D *   (note: depends on token grouping)"),
    ("A * (B + C) / D", "expected: A B C + * D /"),
    ("(A * (B + C))", "expected: A B C + *"),
    # Additional example from the instructions (explicit walk-through)
    ("A*(B+C)/D", "expected: A B C + * D /")
]

for expr, note in examples:
    postfix = infix_to_postfix(expr)
    print(f"Infix: {expr}")
    print("Postfix:", postfix)
    print("->", note)
    print()

Infix: (A+B+C)*D
Postfix: A B + C + D *
-> expected: A B + C + D *   (note: depends on token grouping)

Infix: A * (B + C) / D
Postfix: A B C + * D /
-> expected: A B C + * D /

Infix: (A * (B + C))
Postfix: A B C + *
-> expected: A B C + *

Infix: A*(B+C)/D
Postfix: A B C + * D /
-> expected: A B C + * D /

